# Practical 2: Page-Rank and K-score

[Documentation for sparse matrices](https://docs.scipy.org/doc/scipy/reference/sparse.html)

[Interesting resource](http://www.cs.cmu.edu/~elaw/pagerank.pdf)

## 1. PageRank

Coded in C (see file main_pagerank.c)

# 2. K-core decomposition

In [33]:
progress = -1
max_node = 1004 #found by looping through all edges, and recording max(source, target, max_node)
nb_lines = 1005

### Values to fill out the sparse matrix (one entry per line basically)
### Fills up to 3Go of memory in my computer.
import numpy as np
sources = np.zeros(nb_lines, dtype=int)
targets = np.zeros(nb_lines, dtype=int)
links = np.ones(nb_lines)

import progressbar #to monitor the file reading (takes 2min on my computer)
with open("../TP1/email-Eu-core-department-labels_CLEAN.txt", "r") as file:
    for index, line in enumerate(progressbar.progressbar(file, max_value=nb_lines)): #iterates over "file" basically. Nothing more.
        sources[index], targets[index] = map(int, line.strip("\n").split(" "))

### Storing data as a sparse matrix (framework which performs classic operations only for non-0 elements faster)
from scipy.sparse import coo_matrix, eye
from scipy.sparse.linalg import norm
M = coo_matrix((links, (sources, targets)), shape=(max_node+1, max_node+1)) #Adjacency Matrix

100% (1005 of 1005) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [55]:
#Inspired by https://networkx.github.io/documentation/latest/_modules/networkx/algorithms/core.html
import numpy as np
def core_number(M, sources, targets):
    """Returns the core number for each vertex.The core number of a node is the largest value k of a k-core containing
    that node.

    Parameters
    ----------
    M : adjacency matrix (scipy sparce COO format)
    sources, targets: pairwise elements of the adjacency matrix provided as numpy arrays

    Returns
    -------
    core_number : dictionary
       A dictionary keyed by node to the core number.

    References
    ----------
    .. [1] An O(m) Algorithm for Cores Decomposition of Networks
       Vladimir Batagelj and Matjaz Zaversnik, 2003.
       https://arxiv.org/abs/cs.DS/0310049
    """
    ### Computing degrees
    degrees = np.array(M.sum(axis=1).flatten().tolist()[0], dtype=int) + np.array(M.sum(axis=0).flatten().tolist()[0], dtype=int)
    
    ### Sorting nodes by degree
    nodes = np.array([x for (y,x) in sorted(zip(degrees, np.arange(M.shape[0])))])
    bin_boundaries = [0]
    curr_degree = 0
    for index, vertex in enumerate(nodes):
        if degrees[vertex] > curr_degree:
            bin_boundaries.extend([index] * (degrees[vertex] - curr_degree))
            curr_degree = degrees[vertex]
    node_pos = {vertex: position for position, vertex in enumerate(nodes)}
    
    ### Initializing core numbers as degree
    core_numbers = degrees
    neighbours = {vertex: list(targets[np.where(sources==vertex)]) + list(sources[np.where(targets==vertex)]) for vertex in nodes}
    for vertex1 in nodes:
        for vertex2 in neighbours[vertex1]:
            if core_numbers[vertex2] > core_numbers[vertex1]:
                neighbours[vertex2].remove(vertex1)
                position = node_pos[vertex2]
                bin_start = bin_boundaries[core_numbers[vertex2]]
                node_pos[vertex2] = bin_start
                node_pos[nodes[bin_start]] = position
                nodes[bin_start], nodes[position] = nodes[position], nodes[bin_start]
                bin_boundaries[core_numbers[vertex2]] += 1
                core_numbers[vertex2] -= 1
    return core_numbers

In [69]:
import time as time
start_time = time.time()
core_numbers = core_number(M, sources, targets)
print("k-core decomposition done in {:.2f}s".format(time.time()- start_time))

### Core value of the graph
print("core value is {}".format(max(core_numbers)))

k-core decomposition done in 0.02s
core value is 2
